# 02 Hidden Markov Model

In [4]:
using DelimitedFiles
using MDToolbox
using Plots

## Viterbi algorithm

tests here

\begin{aligned}
L\left(m_{1}, \phi_{1}, d x_{1}, d y_{1}, \ldots, m_{T}, \phi_{T}, d x_{T}, d y_{T}\right)=\sum_{s_{1}=1}^{M} \ldots \sum_{s_{T}=1}^{M} p\left(s_{1}\right) L\left(I \mid m_{1}, \phi_{1}, d x_{1}, d y_{1}\right) \prod_{t=2}^{T} T_{S_{t-1} s_{t}} L\left(I \mid m_{t}, \phi_{t}, d x_{t}, d y_{t}\right)
\end{aligned}

In [5]:
data = readdlm("data/data.dat", ',', Int32)
T = readdlm("data/T.dat", ',', Float64)

3×3 Array{Float64,2}:
 0.1  0.7  0.2
 0.2  0.1  0.7
 0.7  0.2  0.1

## Baum-Welch algorithm (Forward-Backward algorithm)

tests here

\begin{aligned}
L\left(m_{1}, \phi_{1}, d x_{1}, d y_{1}, \ldots, m_{T}, \phi_{T}, d x_{T}, d y_{T}\right)=\sum_{s_{1}=1}^{M} \ldots \sum_{s_{T}=1}^{M} p\left(s_{1}\right) L\left(I \mid m_{1}, \phi_{1}, d x_{1}, d y_{1}\right) \prod_{t=2}^{T} T_{S_{t-1} s_{t}} L\left(I \mid m_{t}, \phi_{t}, d x_{t}, d y_{t}\right)
\end{aligned}

In [6]:
data = readdlm("data/data.dat", ',', Int32)
T = readdlm("data/T.dat", ',', Float64)

3×3 Array{Float64,2}:
 0.1  0.7  0.2
 0.2  0.1  0.7
 0.7  0.2  0.1

In [8]:
emission = readdlm("data/emission.dat", ',', Float64)

3×2 Array{Float64,2}:
 0.9  0.1
 0.6  0.4
 0.1  0.9

In [9]:
pi_i = readdlm("data/pi_i.dat", ',', Float64)
pi_i = pi_i[:]

3-element Array{Float64,1}:
 0.6
 0.3
 0.1

In [11]:
T0 = readdlm("data/T0.dat", ',', Float64)

3×3 Array{Float64,2}:
 0.1  0.1  0.8
 0.8  0.1  0.1
 0.3  0.2  0.5

In [12]:
data_list = [data]

1-element Array{Array{Int32,2},1}:
 [1; 1; … ; 2; 1]

In [16]:
T1, emission1, pi1 = msmbaumwelch(data_list[1], T0, emission, pi_i)

LoadError: DimensionMismatch("tried to assign 3×2 array to 1×3 destination")